In [ ]:
import pandas as pd

import plotly.express as px

from jupyter_dash import JupyterDash

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Read data

In [ ]:
df = pd.read_csv(snakemake.input.fname)
df.head()

# Create Dashboard

In [ ]:
# plotting
def update_figure():
    return px.bar(
        df, x='model', y='accuracy',
        facet_col='dataset',
        barmode='group'
    )

# create app
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Auto-ML Benchmark results'),
    dcc.Graph(figure=update_figure())
])

app.run_server(mode='jupyterlab')